In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np

In [2]:
# !pip install pennylane-qchem


In [3]:
geometry = 'h2.xyz'

In [4]:
charge = 0
multiplicity = 1
basis_set = 'sto-3g'

In [5]:
name = 'h2'

In [8]:
h,qubits = qchem.molecular_hamiltonian(
    name,
    geometry,
    charge=charge,
    mult=multiplicity,
    basis=basis_set,
    active_electrons=2,
    active_orbitals=2,
    mapping='jordan_wigner'
)

print('Number of Qubits: ', qubits)
print('Hamiltonian is: ', h)

Number of Qubits:  4
Hamiltonian is:  (-0.04207897647782188) [I0]
+ (0.17771287465139934) [Z0]
+ (0.1777128746513993) [Z1]
+ (-0.24274280513140484) [Z2]
+ (-0.24274280513140484) [Z3]
+ (0.17059738328801055) [Z0 Z1]
+ (0.04475014401535161) [Y0 X1 X2 Y3]
+ (-0.04475014401535161) [Y0 Y1 X2 X3]
+ (-0.04475014401535161) [X0 X1 Y2 Y3]
+ (0.04475014401535161) [X0 Y1 Y2 X3]
+ (0.12293305056183801) [Z0 Z2]
+ (0.1676831945771896) [Z0 Z3]
+ (0.1676831945771896) [Z1 Z2]
+ (0.12293305056183801) [Z1 Z3]
+ (0.176276408043196) [Z2 Z3]


In [7]:
# from pennylane import qchem

# h, qubits = qchem.molecular_hamiltonian(
#     name='h2',
#     geo_file='h2.xyz',
#     charge=0,
#     mult=1,
#     basis='sto-3g',
#     active_electrons=2,
#     active_orbitals=2
# )

In [9]:
dev = qml.device('default.qubit', wires=qubits)

In [10]:
def circuit(params, wires):
    qml.BasisState(np.array([1, 1, 0, 0]), wires=wires)
    for i in wires:
        qml.Rot(*params[i], wires=i)
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[2, 0])
    qml.CNOT(wires=[3, 1])

In [11]:
cost_fn = qml.ExpvalCost(circuit, h, dev)

In [12]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)
np.random.seed(0)
params = np.random.normal(0, np.pi, (qubits, 3))
print(params)

[[ 5.54193389  1.25713095  3.07479606]
 [ 7.03997361  5.86710646 -3.07020901]
 [ 2.98479079 -0.47550269 -0.32427159]
 [ 1.28993324  0.45252622  4.56873497]]


In [13]:
max_iterations = 200
conv_tol = 1e-06

In [14]:
prev_energy = cost_fn(params)
for n in range(max_iterations):
    params = opt.step(cost_fn, params)
    energy = cost_fn(params)
    conv = np.abs(energy - prev_energy)
    
    if n % 20 == 0:
        print('Iterations= {:}, Energy= {:.8f} Ha'.format(n, energy))
        
    if conv <= conv_tol:
        break
        
    prev_energy = energy
    
print()
print('Final convergence parameter: = {:.8f} Ha'.format(conv))
print('Final value of the ground-state energy = {:.8f} Ha'.format(energy))
print('Accuracy with respect to the FCI energy: {:.8f} Ha ({:.8f} kcal/mol)'.format(
    np.abs(energy - (-1.136189454088)), np.abs(energy - (-1.136189454088))*627.503
    )
)
print()
print('Final circuit parameters = \n', params)

Iterations= 0, Energy= -0.88179557 Ha
Iterations= 20, Energy= -1.13380513 Ha
Iterations= 40, Energy= -1.13558756 Ha
Iterations= 60, Energy= -1.13585794 Ha
Iterations= 80, Energy= -1.13600617 Ha
Iterations= 100, Energy= -1.13608848 Ha
Iterations= 120, Energy= -1.13613394 Ha

Final convergence parameter: = 0.00000099 Ha
Final value of the ground-state energy = -1.13615709 Ha
Accuracy with respect to the FCI energy: 0.00003237 Ha (0.02031093 kcal/mol)

Final circuit parameters = 
 [[ 5.54193389e+00  1.30219523e-08  3.07479606e+00]
 [ 7.03997361e+00  6.28318530e+00 -3.07020901e+00]
 [ 2.98479079e+00 -2.09540998e-01 -4.16893297e-02]
 [ 1.28993324e+00  1.30914606e-12  4.56873497e+00]]
